<a href="https://colab.research.google.com/github/Lorenc1o/DataMining_HackMyRide/blob/main/sanitize_data_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!mkdir main
!gcsfuse --implicit-dirs datamining-ulb main

!ls /content/main

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2426  100  2426    0     0  78258      0 --:--:-- --:--:-- --:--:-- 80866
OK
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 13.3 MB of archives.
After this operation, 30.7 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 124013 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.9_amd64.deb ...
Unpacking gcsfuse (0.41.9) ...
Setting up gcsfuse (0.41.9) ...
2022/12/07 21:09:54.879915 Start gcsfuse/0.41.9 (Go version go1.18.4) for app "" using mount po

In [ ]:
import pandas as pd
import re
import numpy as np
import math
import datetime
import decimal

from glob import glob

decimal.getcontext().prec = 3
TO_MS = decimal.Decimal("1000")

# Helper function
def epoch_ms_to_datetime(epoch_ms: int) -> datetime.datetime:
    return datetime.datetime.fromtimestamp(epoch_ms / int(TO_MS))

In [ ]:
# Extract actual stops
actual_df = pd.read_csv("/content/main/data/actual_stops.csv")
size = actual_df.shape[0]
actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]

actual_df["succession"] = actual_df["succession"].astype(str)
actual_df["numero_lig"] = actual_df["numero_lig"].astype(str)
actual_df["variante"] = actual_df["variante"].astype(str)
actual_stops = set(actual_df["stop_id"])

# Extract Line stop sequence
extracted_lines = {}

delimiters = np.array(["_" for _ in range(size)])
stops_info = (actual_df["numero_lig"].to_numpy() + delimiters +  actual_df["succession"].to_numpy() + delimiters + actual_df["variante"].to_numpy() + delimiters + actual_df["stop_id"].to_numpy()).tolist()

prev_succession = -999999
direction = ""
for stops in stops_info[::-1]:
    info = stops.split("_")
    if int(prev_succession) < int(info[1]):
        direction = info[3]
    
    data = extracted_lines.setdefault(info[0], {})
    
    succession_info = data.setdefault(info[3], {})
    succession_info[direction] = info[1]
    
    data[info[3]] = succession_info
    extracted_lines[info[0]] = data
    
    line_direction = data.setdefault("direction", {})
    visited_stops = line_direction.setdefault(direction, set())
    visited_stops.add(info[3])
    
    line_direction[direction] = visited_stops
    data["direction"] = line_direction
    
    prev_succession = info[1]

'''
Pair stops with the succession line, 
 1. if there's multiple sucession line check whether direction is the same
 2. If not, try to find direction id in the direction stops
 3. otherwise set sucession to 99999
'''

lines = []
names = set()
for file in glob("/content/main/ordered_lines/*.csv"):
  name = file.strip().split("/")[4].split("_")[0]
  names.add(name)

print(names)
for file in glob("/content/main/Lines_vehiclePositions/*.csv"):
    if "null" in file or file in names:
      continue

    df = pd.read_csv(file)
    df["lineID"] = df["lineID"].astype(str)
    df["directionID"] = df["directionID"].astype(str)
    df["pointID"] = df["pointID"].astype(str).apply(lambda x: x.strip().split(" ")[0])
    
    delimiters = np.array(["_" for _ in range(df.shape[0])])
    stops_infos = (df["lineID"].to_numpy() + delimiters + df["directionID"].to_numpy() + delimiters + df["pointID"].to_numpy() + delimiters + df.index.astype(str).values).tolist()
    
    succession_arr = []
    for stop in stops_infos:
        info = stop.split("_")
        data = extracted_lines[info[0]]
        
        succession = 999999
        if info[2] not in data:
            succession_arr.append(succession)
            continue
        
        succession_info = data[info[2]]
        if info[1] not in succession_info:
            succession_arr.append(succession)
            continue
        
        succession = succession_info[info[1]]
        succession_arr.append(succession)
    
    df["succession"] = np.array(succession_arr)
    
    df["lineID"] = df["lineID"].astype(int)
    df["directionID"] = df["directionID"].astype(int)
    df["pointID"] = df["pointID"].astype(int)
    df["succession"] = df["succession"].astype(int)

    name = file.strip().split("/")[4].split("_")[0]
    df.sort_values(["directionID", "time",  "succession",], ascending=[True, True, True]).to_csv(f"/content/main/ordered_lines/{name}_ordered.csv")
    

{'Line17', 'Line27', 'Line3', 'Line36', 'Line48', 'Line6', 'Line12', 'Line92', 'Line61', 'Line82', 'Line76', 'Line70', 'Line93', 'Line95', 'Line29', 'Line39', 'Line1', 'Line21', 'Line51', 'Line56', 'Line88', 'Line33', 'Line64', 'Line80', 'Line62', 'Line28', 'Line25', 'Line54', 'Line98', 'Line49', 'Line41', 'Line38', 'Line20', 'Line14', 'Line19', 'Line2', 'Line65', 'Line78', 'Line81', 'Line71', 'Line47', 'Line69', 'Line57', 'Line55', 'Line63', 'Line34', 'Line7', 'Line42', 'Line74', 'Line86', 'Line59', 'Line75', 'Line89', 'Line50', 'Line79', 'Line87', 'Line53', 'Line46', 'Line5', 'Line4', 'Line8', 'Line97', 'Line58', 'Line66', 'Line43', 'Line13', 'Line72', 'Line83', 'Line37', 'Line44', 'Line60', 'Line45', 'Line77', 'Line9'}


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
'''
Normalize the distance
 The algorithm will work as follow:
 1. We will change the distance for non-zero distance point by subtracting the time with how much time passed since the bus in the 0 distance of the point
 2. We will use the bus/tram average speed to calculate the distance passed
'''

AVERAGE_SPEED = decimal.Decimal("4.667") # in metre/second

# TODO: Changes it for every line
line_df = pd.read_csv("line_71_sucession.csv")

# Remove outlier
line_df = line_df[line_df["succession"] != 999999]

columns = ["time", "lineID", "directionID", "distancefromPoint", "pointID", "succession"]

size = line_df.shape[0]
delimiters = np.array(["_" for _ in range(size)])

line_infos = None
for idx, column in enumerate(columns):
    line_df[column] = line_df[column].astype(str)
    if line_infos is None:
        line_infos = line_df[column].to_numpy() + delimiters
    elif line_infos is not None and idx < len(columns)-1:
        line_infos += line_df[column].to_numpy() + delimiters
    else:
        line_infos += line_df[column].to_numpy()

normalized_timestamp = []
normalized_distance = []
for line_info in line_infos:
    line = line_info.split("_")

    # Define var
    timestamp = int(line[0])
    distance = decimal.Decimal(line[3])
    
    if distance == 0:
        normalized_timestamp.append(epoch_ms_to_datetime(timestamp))
        normalized_distance.append(distance)
        continue
        
    # Normalized the timestamp
    time_passed = distance / AVERAGE_SPEED
    timestamp -= (int(time_passed*TO_MS))
    
    normalized_timestamp.append(epoch_ms_to_datetime(timestamp))
    normalized_distance.append(math.floor(abs(distance - (AVERAGE_SPEED*time_passed))))


line_df["normalized_timestamp"] = np.array(normalized_timestamp)
line_df["normalized_distance"] = np.array(normalized_distance)

for idx, column in enumerate(columns):
    line_df[column] = line_df[column].astype(int)


line_df["time"] = line_df["time"].apply(lambda x: epoch_ms_to_datetime(x))
line_df.drop('Unnamed: 0', axis=1, inplace=True)
line_df.sort_values(["directionID", "succession", "normalized_timestamp"], ascending=[True, True, True]).to_csv("line_71_normalized.csv")

/content
